In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import FastText
from pythainlp.util import normalize
from pythainlp.spell import correct
from PIL import Image, ImageTk
import os

# ===== โหลดข้อมูลจาก CSV =====
df = pd.read_csv("fashion_products_thai.csv")

# ===== ฟังก์ชันประมวลผลข้อความ =====
def clean_text(text):
    text = str(text).lower().strip()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r'\d+', '', text)
    return text

def normalize_input(text):
    text = normalize(text)
    text = re.sub(r"\s+", "", text)
    return correct(text)

# ===== เตรียมข้อมูล =====
df["cleaned_product"] = df["product"].apply(clean_text)
df["cleaned_description"] = df["description"].apply(clean_text)

# ===== เตรียม TF-IDF และ FastText =====
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["cleaned_description"])

sentences = [row.split() for row in df["cleaned_description"]]
ft_model = FastText(vector_size=100, window=5, min_count=1, workers=4)
ft_model.build_vocab(sentences)
ft_model.train(sentences, total_examples=len(sentences), epochs=10)

# ===== ฟังก์ชันระบบแนะนำสินค้า =====
def validate_input(text):
    text = text.strip()
    if len(set(text)) <= 2:
        return False
    return True

def get_most_similar_word(user_input):
    try:
        similar_words = ft_model.wv.most_similar(user_input, topn=5)
        return [word for word, _ in similar_words]
    except:
        return []

def recommend_product(product_name):
    product_name = clean_text(normalize_input(product_name))

    matching_products = df[df["cleaned_description"].str.contains(product_name, na=False) |
                           df["cleaned_product"].str.contains(product_name, na=False)]

    if not matching_products.empty:
        return ("", list(zip(matching_products["product"], matching_products["description"], matching_products["image_path"])))

    similar_words = get_most_similar_word(product_name)
    if similar_words:
        suggestion_text = f"ไม่พบสินค้านั้น\nคุณอาจหมายถึง: {', '.join(similar_words)}"
        similar_matches = df[df["cleaned_description"].apply(lambda x: any(word in x for word in similar_words)) |
                             df["cleaned_product"].apply(lambda x: any(word in x for word in similar_words))]
        return (suggestion_text, list(zip(similar_matches["product"], similar_matches["description"], similar_matches["image_path"][:5])))
    else:
        return ("ไม่พบสินค้านั้นและไม่มีคำที่ใกล้เคียง", [])

# ===== ฟังก์ชันแสดงรูปภาพ =====
def show_image(image_path):
    try:
        if not os.path.isfile(image_path):
            raise FileNotFoundError
        img = Image.open(image_path)
        img = img.resize((150, 150))
        photo = ImageTk.PhotoImage(img)
        image_label.configure(image=photo, text="")
        image_label.image = photo
    except Exception:
        image_label.configure(image='', text="ไม่พบรูปภาพ", font=("Sarabun", 10))

# ===== ฟังก์ชันเมื่อกดปุ่ม =====
def on_submit():
    product_name = entry.get().strip()
    if not product_name:
        messagebox.showwarning("แจ้งเตือน", "กรุณากรอกชื่อสินค้า")
        return

    for widget in product_frame.winfo_children():
        widget.destroy()
    image_label.configure(image='', text="")
    result_text.set("")

    processed_name = clean_text(normalize_input(product_name))
    if not validate_input(processed_name):
        result_text.set("ไม่มีสินค้าแนะนำ")
        return

    suggestion, recommendations = recommend_product(product_name)

    if suggestion:
        result_text.set(suggestion)
    else:
        result_text.set("")

    display_product_cards(recommendations[:5])  # แสดงแค่ 5 อัน

# ===== ฟังก์ชันแสดงสินค้าในรูปแบบการ์ด =====
def display_product_cards(recommendations):
    for widget in product_frame.winfo_children():
        widget.destroy()

    if not recommendations:
        result_text.set("ไม่มีสินค้าแนะนำ")
        return

    row = 0
    col = 0
    for product, description, image_path in recommendations:
        create_product_card(product_frame, product, description, image_path, row, col)
        col += 1
        if col == 2:
            col = 0
            row += 1

def create_product_card(parent, product, description, image_path, row, col):
    card = tk.Frame(parent, bd=1, relief=tk.SOLID, padx=10, pady=10)
    card.grid(row=row, column=col, padx=10, pady=10)

    try:
        img = Image.open(image_path)
        img = img.resize((120, 120))
        photo = ImageTk.PhotoImage(img)
    except:
        photo = ImageTk.PhotoImage(Image.new('RGB', (120, 120), color='gray'))

    img_label = tk.Label(card, image=photo)
    img_label.image = photo
    img_label.pack()

    name_label = tk.Label(card, text=product, font=("Sarabun", 12, "bold"), wraplength=120)
    name_label.pack(pady=(5, 0))

    description_label = tk.Label(card, text=description, font=("Sarabun", 10), wraplength=120)
    description_label.pack(pady=(5, 0))

# ===== GUI =====
root = tk.Tk()
root.title("\U0001F6D2 ระบบแนะนำสินค้า")
root.geometry("600x700")
root.configure(bg="#FFFFFF")

title_label = tk.Label(root, text="ระบบแนะนำสินค้า", font=("Sarabun", 18, "bold"), fg="#000000", bg="#FFFFFF")
title_label.pack(pady=(15, 5))

tk.Label(root, text="กรุณากรอกชื่อสินค้า:", font=("Sarabun", 12), bg="#FFFFFF", fg="#000000").pack(pady=(10, 0))

entry = tk.Entry(root, width=35, font=("Sarabun", 12), fg="#000000", bg="#FFFFFF")
entry.pack(pady=8, ipady=5)

btn = tk.Button(root, text="\U0001F50D แนะนำสินค้า", command=on_submit, bg="#D3D3D3", fg="#000000",
                font=("Sarabun", 12, "bold"), activebackground="#E64A19", relief="flat", padx=10, pady=5)
btn.pack(pady=10)

result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, justify="left", wraplength=350, font=("Sarabun", 12), bg="#FFFFFF", fg="#000000")
result_label.pack(pady=10)

product_frame = tk.Frame(root, bg="#FFFFFF")
product_frame.pack(pady=10)

image_label = tk.Label(root, bg="#FFFFFF")
image_label.pack()

root.mainloop()


2025-04-23 22:48:45.305 Python[81276:4338705] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-23 22:48:45.305 Python[81276:4338705] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-04-23 22:48:48.870 Python[81276:4338705] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 

In [1]:
echo "# NLP" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/Skyisblue2003/NLP.git
git push -u origin main

SyntaxError: invalid syntax (3386715621.py, line 1)